In [ ]:
%pip install crewai crewai-tools poetry vllm
!curl -fsSL https://ollama.com/install.sh | sh
!pip install pymupdf langchain-community langchain-core langgraph faiss-cpu
!pip install langchain --upgrade

In [ ]:
!nohup ollama serve > ollama.log &
!nohup ollama run llava:7b &

nohup: redirecting stderr to stdout
nohup: appending output to 'nohup.out'


In [ ]:
import pymupdf  # PyMuPDF
import os
import zipfile

# Caminho do arquivo ZIP enviado pelo usuário
zip_path = "RAG.zip"
extracted_path = "extracted_pdfs"

# Extraindo os PDFs
os.makedirs(extracted_path, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

# Função para extrair texto de PDFs
def extract_text_from_pdfs(pdf_folder):
    text_data = ""
    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            doc = pymupdf.open(pdf_path)
            for page in doc:
                text_data += page.get_text("text") + "\n\n"
    return text_data

# Extraindo texto dos PDFs
medical_texts = extract_text_from_pdfs(extracted_path)

# Salvando em um arquivo de texto
medical_texts_path = "medical_texts.txt"
with open(medical_texts_path, "w", encoding="utf-8") as f:
    f.write(medical_texts)

# Retornar o caminho do arquivo salvo
medical_texts_path


In [ ]:
# LangChain supports many other chat models. Here, we're using Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters
llm = ChatOllama(model="llava:7b")
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# using LangChain Expressive Language chain syntax
# learn more about the LCEL on
# /docs/expression_language/why
chain = prompt | llm | StrOutputParser()

# for brevity, response is printed in terminal
# You can use LangServe to deploy your application for
# production
print(chain.invoke({"topic": "Space travel"}))

<ipython-input-3-8b2e26c1fe6e>:8: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llava:7b")


 Why did the astronaut go to space?

To explore strange new worlds, to seek out new life and new civilizations, to boldly go where no one has gone before. 


In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain.schema import SystemMessage, HumanMessage
from langchain_community.vectorstores.faiss import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langgraph.graph import StateGraph, START, END
from langchain.pydantic_v1 import BaseModel


# Carregar embeddings do Sentence-Transformers
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Carregar e processar documentos médicos (já extraídos dos PDFs)
loader = TextLoader("medical_texts.txt")  # Arquivo consolidado com informações médicas extraídas
texts = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(texts)

# Criar banco vetorial FAISS
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

# Configurar modelo LLM
llm = ChatOllama(model="llava:7b")  # Trocar conforme necessário

# Configurar agentes
class DiagnosticAgent:
    def __init__(self, llm):
        self.llm = llm

    def analyze_lesion(self, state):
        # Simulação da análise de imagem segmentada
        return {"diagnosis": self.llm.invoke(state.image_data).content}#"Lesão apresenta pigmentação irregular e bordas assimétricas. Possível risco moderado."}

class PrognosticAgent:
    def __init__(self, llm):
        self.llm = llm

    def assess_risk(self, state):
      return {"prognosis": "Baseado no diagnóstico, a lesão tem 60% de chance de ser maligna."}

class SummaryAgent:
    def __init__(self, llm):
        self.llm = llm

    def summarize(self, state):
        return {
            "diagnosis": state.diagnosis,
            "prognosis": state.prognosis,
            "validation": "Recomenda-se biópsia para confirmação."
        }

# Configurar agente crítico
class CriticalReviewAgent:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm

    def validate_diagnosis(self, state):
        retrieved_docs = self.retriever.invoke(state.diagnosis)
        return {"final_report": f"Confirmação baseada em literatura médica: {retrieved_docs[0].page_content[:200]}..."}


# Definir o esquema de estado inicial
class GraphState(BaseModel):
    image_data: str
    diagnosis: str = None
    prognosis: str = None
    validation: str = None
    final_report: str = None


# Criar fluxo no LangGraph
graph = StateGraph(GraphState)

graph.add_node("diagnostic", DiagnosticAgent(llm).analyze_lesion)
graph.add_node("prognostic", PrognosticAgent(llm).assess_risk)
graph.add_node("summary", SummaryAgent(llm).summarize)
graph.add_node("critical_review", CriticalReviewAgent(retriever, llm).validate_diagnosis)

# Definir conexões do fluxo
graph.add_edge(START, "diagnostic")
graph.add_edge("diagnostic", "prognostic")
graph.add_edge("prognostic", "summary")
graph.add_edge("summary", "critical_review")
graph.add_edge("critical_review", END)

g = graph.compile()

# Executar o fluxo de teste
output = g.stream({"image_data": "imagem_segmentada_mock"}, stream_mode="values")
print(output)


<generator object Pregel.stream at 0x7db587b9a8f0>


In [ ]:
a = llm.invoke("imagem_segmentada_mock")

In [ ]:
a

AIMessage(content=' Imagem segmentada é a tarefa de separar um objeto ou parte de uma imagem do fundo da imagem. Esta tarefa é comum em aplicações como detecção de objetos, reconhecimento facial e análise de imagens médicas.\n\nPara executar essa tarefa, geralmente usam-se algoritmos de segmentação de imagens. Alguns desses algoritmos incluem o metodo da máscara, o método do contorno e o método da região de interesse (ROI).\n\nO metodo da máscara consiste em dividir a imagem em blocos ou matrizes pequenas e analisar cada uma das peças, enquanto o metodo do contorno analisa as bordas da imagem. O método da ROI envolve identificar a parte da imagem que deve ser segmentada e realizar a segmentação apenas nessa parte.\n\nExistam várias bibliotecas e ferramentas disponíveis para facilitar o processo de segmentação de imagens, como OpenCV, Python, C++ e Java. ', additional_kwargs={}, response_metadata={'model': 'llava:7b', 'created_at': '2025-02-26T10:54:47.403378527Z', 'message': {'role': '

In [ ]:
output = g.stream({"image_data": "imagem_segmentada_mock"}, stream_mode="values")
# print(output)

for event in g.stream({"image_data": "imagem_segmentada_mock"}, stream_mode="values"):
  print(event)

{'image_data': 'imagem_segmentada_mock'}
{'image_data': 'imagem_segmentada_mock', 'diagnosis': ' Desculpe, não consigo fornecer uma imagem específica sem mais informações. No entanto, posso descrever a funcionalidade geral da segmentação de imagens para você.\n\nA segmentação de imagens é um processo que envolve dividir uma imagem em partes menores e definir para cada parte uma classificação ou atributo. Por exemplo, podemos usar segmentação para identificar objetos dentro de uma imagem, destacar a região de interesse ou separar estruturas ou elementos da imagem.\n\nHá vários métodos e algoritmos para realizar essa tarefa, como:\n\n1. Segmentação por thresholding: onde se define um valor de limiar (threshold) e toda a região acima desse valor é considerada como uma parte do objeto e abaixo é ignorado.\n2. Segmentação por edge detection: onde são identificados os bordos das partes da imagem e as regiões interiores são definidas.\n3. Segmentação por clustering: onde se utiliza algoritmos